In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tensorflow.keras import callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input , Dense ,Dropout
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import Sequence
from scipy.sparse import vstack
import bz2
import numpy as np
import pickle
import mmap 
import re

In [23]:

MAX_FEATURES = 12000
sequences = Input(shape=(255,))
embedded = layers.Embedding(MAX_FEATURES, 32)(sequences)
x = layers.Conv1D(64, 3, activation='relu')(embedded)
x = layers.BatchNormalization()(x)
x = layers.MaxPool1D(3)(x)
x = layers.Conv1D(32, 5, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool1D(5)(x)
x = layers.Conv1D(16, 5, activation='relu')(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)

In [24]:
tfidf = Input(shape=(1000,))
x1 = layers.Dense(128, activation='relu')(tfidf)
x1 = layers.Dense(64, activation='relu')(x1)
     

In [25]:
merged = layers.concatenate([x, x1])


In [26]:
dense1 = Dense(256, activation='relu')(merged)
dense1 = Dropout(0.1)(dense1)
dense1 = Dense(32, activation='relu')(dense1)
outputs = Dense(1, activation='sigmoid')(dense1)

In [27]:
model = Model(inputs=[sequences, tfidf], outputs=outputs)
     

mch = callbacks.ModelCheckpoint('../models/fianl-model-dualchanal.h5' , monitor='accuracy' , mode ='max' , save_best_only=True)
     

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [8]:
class DualChannelDataGenerator(Sequence):
    def __init__(self, dataset_size , seq_x_file, tfidf_x_file, y_file, batch_size):
      self.dataset_size = dataset_size
      with open(seq_x_file, 'rb') as f:
          mm = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
          self.seq_x = np.load(mm, allow_pickle=True)
      with open(tfidf_x_file, 'rb') as f:
          mm = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
          self.tfidf_x = np.load(mm, allow_pickle=True)
      with open(y_file, 'rb') as f:
          mm = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
          self.y = np.load(mm, allow_pickle=True)
      self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(self.dataset_size/ float(self.batch_size)))

    def __getitem__(self, idx):
        batch_seq_x = self.seq_x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_tfidf_x = self.tfidf_x[idx * self.batch_size:(idx + 1) * self.batch_size].toarray()
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return [batch_seq_x, batch_tfidf_x], batch_y



In [9]:
with open('/content/drive/MyDrive/persian-sentiment-analysis/X_train_tfidf1.pickle' , 'rb') as f:
  tfidf_1 = pickle.load(f)



with open('/content/X_train_tfidf.pickle' , 'wb') as f:
  pickle.dump(tfidf_1 , f)

del tfidf_1


In [10]:
with open('/content/drive/MyDrive/persian-sentiment-analysis/X_test_tfidf.pickle' , 'rb') as f:
  tfidf = pickle.load(f)

with open('/content/X_test_tfidf.pickle' , 'wb') as f:
  pickle.dump(tfidf , f)

del tfidf


In [11]:
with open('/content/drive/MyDrive/persian-sentiment-analysis/train_pad_sequences1.pickle' , 'rb') as f:
  seq_1 = pickle.load(f)

with open('/content/drive/MyDrive/persian-sentiment-analysis/train_pad_sequences2.pickle' , 'rb') as f:
  seq_2 = pickle.load(f)

with open('/content/train_pad_sequences.pickle' , 'wb') as f:
  pickle.dump(np.concatenate((seq_1 , seq_2), axis=0) , f)

del seq_1
del seq_2

In [12]:
with open('/content/drive/MyDrive/persian-sentiment-analysis/test_pad_sequences.pickle' , 'rb') as f:
  seq_test = pickle.load(f)

with open('/content/test_pad_sequences.pickle' , 'wb') as f:
  pickle.dump(seq_test , f)

del seq_test
     

In [13]:
with open('/content/drive/MyDrive/persian-sentiment-analysis/train_labels.pickle' , 'rb') as f:
  train_labels = pickle.load(f)

with open('/content/train_labels.pickle' , 'wb') as f:
  pickle.dump(train_labels, f)

del train_labels

In [14]:
with open('/content/drive/MyDrive/persian-sentiment-analysis/test_labels.pickle' , 'rb') as f:
  test_labels = pickle.load(f)

with open('/content/test_labels.pickle' , 'wb') as f:
  pickle.dump(test_labels , f)

del test_labels
     

In [15]:
batch_size = 128
dataset_size_train = 3600000
dataset_size_test = 400000
seq_path_train = '/content/train_pad_sequences.pickle'
seq_path_test =  '/content/test_pad_sequences.pickle'
tfidf_train = '/content/X_train_tfidf.pickle'
tfidf_test = '/content/X_test_tfidf.pickle'
train_labels = '/content/train_labels.pickle'
test_labels = '/content/test_labels.pickle'

In [16]:
train_generator = DualChannelDataGenerator(dataset_size_train ,seq_path_train , tfidf_train , train_labels , batch_size)


In [17]:
test_generator = DualChannelDataGenerator(dataset_size_test ,seq_path_test , tfidf_test , test_labels , batch_size)


In [28]:
model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=10 , validation_data=test_generator, validation_steps=len(test_generator),callbacks=[mch])

Epoch 1/10
28125/28125 [==============================] - 564s 20ms/step - loss: 0.1609 - accuracy: 0.9382 - val_loss: 0.1446 - val_accuracy: 0.9466
Epoch 2/10
28125/28125 [==============================] - 339s 12ms/step - loss: 0.1293 - accuracy: 0.9525 - val_loss: 0.1368 - val_accuracy: 0.9497
Epoch 3/10
28125/28125 [==============================] - 340s 12ms/step - loss: 0.1172 - accuracy: 0.9576 - val_loss: 0.1326 - val_accuracy: 0.9512
Epoch 4/10
28125/28125 [==============================] - 339s 12ms/step - loss: 0.1079 - accuracy: 0.9616 - val_loss: 0.1350 - val_accuracy: 0.9509
Epoch 5/10
28125/28125 [==============================] - 337s 12ms/step - loss: 0.1003 - accuracy: 0.9648 - val_loss: 0.1388 - val_accuracy: 0.9500
Epoch 6/10
28125/28125 [==============================] - 336s 12ms/step - loss: 0.0937 - accuracy: 0.9674 - val_loss: 0.1454 - val_accuracy: 0.9484
Epoch 7/10
28125/28125 [==============================] - 347s 12ms/step - loss: 0.0880 - accuracy: 0.9698